In [ ]:
import pandas as pd
import numpy as np

data_all = np.array(pd.read_csv('/media/bruno/f71fc4fd-abcd-4a4b-9fb1-6a8899041c1c/EU-2019/EURUSD_1m_all.csv',header=None))
m15 = []
m30 = []
h1 = []
h4 = []
d1 = []
hours = [
 '00:00', '01:00', '02:00', '03:00',
 '05:00', '06:00', '07:00', '09:00',
 '10:00', '11:00', '13:00', '14:00',
 '15:00', '17:00', '18:00', '19:00',
 '21:00', '22:00', '23:00']

four_hours = ['04:00','08:00','12:00','16:00','20:00']


for i in range(len(data_all)):
    
    if str(data_all[i][1]).endswith('00'):
        
        if str(data_all[i][1])  == '00:00':
            
            d1.append(i)
            h4.append(i)
            h1.append(i)
            m30.append(i)
            m15.append(i)
            continue
        elif str(data_all[i][1]) in hours:
            h1.append(i)
            m30.append(i)
            m15.append(i)
            continue
        elif str(data_all[i][1]) in four_hours:
            h1.append(i)
            h4.append(i)
            m30.append(i)
            continue
    elif str(data_all[i][1]).endswith('15'):
        m15.append(i)
        continue
    elif str(data_all[i][1]).endswith('30'):
        m30.append(i)
        m15.append(i)
        continue
    elif str(data_all[i][1]).endswith('45'):
        m15.append(i)
        continue



In [ ]:
class Row:
    def __init__(self, date, time, open_p, high, low, close, volume, state):
        self.date = date
        self.time = time
        self.open_p = float(open_p)
        self.high = float(high)
        self.low = float(low)
        self.close = float(close)
        self.volume = int(volume)
        self.state = state

    def makeRecord(self):
        return list(
            [
                self.date,
                self.time,
                self.open_p,
                self.high,
                self.low,
                self.close,
                self.volume,
                self.state,
            ]
        )


In [113]:
def getMax(column_index, start, stop):
    values = [float(data_all[i][column_index]) for i in range(start, stop)]
    return max(values)

def getMin(column_index, start, stop):
    values = [float(data_all[i][column_index]) for i in range(start, stop)]
    st = ','.join(str(values))
    return min(values)

def getSum(column_index, start, stop):
    values = [int(data_all[i][column_index]) for i in range(start, stop)]

    return sum(values)


So now that you have lists of positions for each time interval, use those to recalculate OHLCVS(indices 2,3,4,5,6,7)
<!-- for each index listed in h1,h4,etc look for a corresponding element in data_all -->
<!-- open = same open as that index -->
<!-- close = close of index prior to interval index -->
<!-- high = highest value data_all[index][3] in range(curr_interval_index, next_interval_index) -->
<!-- low =  lowest value data_all[index][3] in range(curr_interval_index, next_interval_index)-->
<!-- volume = sum of data_all[index][6] in range(curr_interval_index, next_interval_index) -->
<!-- state = [0 if open==close, 1 if close>open, -1 if close<open] -->

In [116]:
m30_data = [['date','time','open','high','low','close','volume','state']]
for i in range(len(m30)):
    
    if (i+1) == len(m30):
        break
    row = Row(
        date=data_all[m30[i]][0],
        time=data_all[m30[i]][1],
        open_p=data_all[m30[i]][2],
        high=getMax(3, m30[i], m30[i + 1]),
        low=getMin(4, m30[i], m30[i + 1]),
        close=data_all[(m30[i+1])-1][5],
        volume=getSum(6, m30[i], m30[i + 1]),
        state = 0
    )
    if row.close > row.open_p:
        row.state = 1
    elif row.close < row.open_p:
        row.state = -1
    else:
        row.state = 0
    m30_data.append(row.makeRecord())


[2, 8, 1, 3, 8, 2, 2, 6, 16, 23, 28, 11, 17, 12, 18, 7, 1, 11, 9, 19, 17, 25, 26, 25, 18, 9, 28]
[21, 12, 17, 40, 17, 13, 10, 20, 6, 11, 25, 30, 18, 24, 13, 47, 26, 26, 32, 38, 55, 51, 53, 10, 31, 21, 23, 31, 53, 44]
[352, 103, 38, 22, 25, 36, 21, 18, 12, 24, 75, 35, 87, 99, 34, 23, 36, 101, 98, 69, 92, 50, 62, 49, 49, 57, 37, 42, 18, 23]
[24, 36, 37, 27, 29, 21, 28, 45, 31, 9, 23, 20, 12, 32, 18, 12, 26, 23, 18, 37, 38, 21, 19, 26, 25, 14, 10, 16, 24, 10]
[56, 38, 37, 24, 36, 126, 100, 35, 25, 32, 47, 34, 20, 25, 27, 30, 26, 43, 38, 24, 55, 25, 38, 11, 7, 37, 21, 19, 13, 49]
[101, 44, 68, 36, 28, 45, 41, 26, 23, 28, 28, 19, 25, 45, 23, 25, 31, 34, 8, 41, 7, 25, 52, 4, 36, 28, 34, 4, 8]
[22, 43, 52, 73, 73, 64, 35, 47, 30, 39, 33, 45, 68, 71, 35, 38, 27, 24, 28, 44, 23, 19, 32, 14, 7, 20, 17, 24, 26, 37]
[57, 33, 28, 48, 73, 37, 64, 50, 41, 56, 36, 67, 27, 16, 22, 86, 45, 58, 49, 78, 124, 80, 77, 43, 82, 87, 42, 36, 53, 34]
[107, 93, 45, 47, 28, 25, 40, 18, 47, 32, 23, 52, 47, 82, 36, 

In [117]:
m30_data = pd.DataFrame(m30_data)
m30_data.to_csv('EURUSD_30m_all.csv',header=False)